In [3]:
import requests, json
import pandas as pd
import numpy as np
import seaborn
from bs4 import BeautifulSoup
%matplotlib inline

# Exercice 4

We took the correlation matrix of the merge dataframe and unstacked it. We considered that the correlation was :
- strong above 0.7
- moderate between 0.7 and 0.5
- weak between 0.5 and 0.3
- inexistant below 0.3

In [66]:
#correlation matrix, with absolute values only
correlations = merged_df.corr().abs()

#take only the upper triangle to no get duplicates
correlations = correlations.where(np.triu(np.ones(correlations.shape)).astype(np.bool))

#sort values
correlations = correlations.unstack().sort_values(ascending=False)

#  divies by strong/moderate/weak/no correlations
strong_corr = correlations[(correlations < 1) & (correlations >= 0.7)]
moderate_corr = correlations[(correlations < 0.7) & (correlations >= 0.5)]
weak_corr = correlations[(correlations < 0.5) & (correlations >= 0.3)]
no_corr = correlations[(correlations < 0.3)]

We can see that strongly correlated values are the same values issued from the two different sources, like total number of students in the QS and total number of students in THE, which seems logical.

In [67]:
strong_corr

nbstudents_all_universitiesTimes             nbstudents_all_universitiesShanghai               0.983463
ratioIntStudentmem_universitiesTimes         ratioIntStudentmem_universitiesShanghai           0.936593
nbstudents_int_universitiesTimes             nbstudents_int_universitiesShanghai               0.887394
nbfaculty_all_universitiesTimes              nbfaculty_all_universitiesShanghai                0.737562
ratioFacultymemStudentmem_universitiesTimes  ratioFacultymemStudentmem_universitiesShanghai    0.715505
rank_universitiesTimes                       rank_universitiesShanghai                         0.704325
dtype: float64

In moderate correlation, it's interesting to see that the rank has a score or 0.5 with the number of international faculty members, but correlation doesn't give us causation therefore we can't define if a better university will have more international faculty members or if it's the other way around, or both.
The other values of this list show us that the number of students is proportionnal to the number of faculty members, and that the number of internationals (student and faculty members) is correlated to the total numbers (student and faculty members).

In [68]:
moderate_corr

nbfaculty_int                            nbfaculty_all_universitiesShanghai     0.647662
nbstudents_int_universitiesShanghai      nbstudents_all_universitiesShanghai    0.629987
nbstudents_int_universitiesTimes         nbstudents_all_universitiesShanghai    0.592219
nbstudents_all_universitiesShanghai      nbfaculty_all_universitiesShanghai     0.589139
nbstudents_all_universitiesTimes         nbfaculty_all_universitiesShanghai     0.586477
                                         nbstudents_int_universitiesShanghai    0.583989
nbstudents_int_universitiesShanghai      nbfaculty_int                          0.579686
nbstudents_int_universitiesTimes         nbstudents_all_universitiesTimes       0.553896
nbstudents_all_universitiesTimes         nbfaculty_all_universitiesTimes        0.547380
ratioIntStudentmem_universitiesShanghai  nbstudents_int_universitiesShanghai    0.535795
ratioIntStudentmem_universitiesTimes     nbstudents_int_universitiesTimes       0.507608
rank_universitiesShan

The rest of the values show us the lowest correlations. What is interesting to look, is that the rank has low correlation with the number of faculty members (total and international), almost no corrlation with the number of students (total and international) and the ratio faculty members/student. 

In [107]:
print(weak_corr)
print(no_corr)

nbfaculty_all_universitiesTimes                 nbstudents_all_universitiesShanghai               0.487660
nbstudents_int_universitiesTimes                ratioIntStudentmem_universitiesShanghai           0.484099
ratioFacultymemStudentmem_universitiesTimes     nbfaculty_all_universitiesTimes                   0.456813
nbstudents_int_universitiesTimes                nbfaculty_int                                     0.439998
nbstudents_int_universitiesShanghai             nbfaculty_all_universitiesShanghai                0.417124
ratioFacultymemStudentmem_universitiesShanghai  rank_universitiesShanghai                         0.400492
rank_universitiesTimes                          nbfaculty_all_universitiesShanghai                0.395698
ratioFacultymemStudentmem_universitiesShanghai  nbstudents_all_universitiesShanghai               0.395150
ratioFacultymemStudentmem_universitiesTimes     nbstudents_all_universitiesShanghai               0.389592
nbstudents_int_universitiesTimes     

# Exercice 5

For the merging, we considered that both rankings had the same relevance, therefore we could simply take the mean (or the sum, since it won't change the order).

In [143]:
pd.options.mode.chained_assignment = None  # default='warn'

# take only universities that were on both rankings
merged_without_nan = merged_df.dropna()

# sum the rank
merged_without_nan['rank_sum'] = merged_without_nan[['rank_universitiesShanghai','rank_universitiesTimes']].sum(axis=1)

# sort them
final_ranking = merged_without_nan.sort_values(['rank_sum'], ascending=True).reset_index(drop=True)

# increment index by one so it starts at 1
final_ranking.index +=1

final_ranking[['rank_sum', 'title']]

,rank_sum,title
1,5.0,Stanford University
2,7.0,University of Oxford
3,7.0,University of Cambridge
4,9.0,Harvard University
5,16.0,Imperial College London
6,18.0,University of Chicago
7,20.0,Princeton University
8,28.0,Yale University
9,29.0,University of Pennsylvania
10,30.0,Johns Hopkins University
